<a href="https://colab.research.google.com/github/SadafShafi/Sadaf-Avatar/blob/main/TTS_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TensorFlowTTS 
!pip install git+https://github.com/repodiac/german_transliterate.git#egg=german_transliterate 

In [1]:
# !pip uninstall tensorflow-estimator

# !pip install tensorflow-estimator==2.1

In [ ]:
!pip install --upgrade tensorflow

In [4]:
import numpy as np
import soundfile as sf
import yaml

import tensorflow as tf

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")


# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")


# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


Downloading:   0%|          | 0.00/125M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

In [8]:
text = " tremendous mix my brother, without a doubt he has just earned a little place in my favorites playlist. Congratulations.."

In [9]:
input_ids = processor.text_to_sequence(text)
# fastspeech inference

mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
    
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),

    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),

    speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),

    f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),

    energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    
)

# melgan inference
audio_before = mb_melgan.inference(mel_before)[0, :, 0]
audio_after = mb_melgan.inference(mel_after)[0, :, 0]

# save to file
sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")

In [10]:
from IPython.display import Audio
# signal = read_audio("/content/audio_after.wav").squeeze()
Audio("/content/audio_after.wav", rate=16000)